In [1]:
import os
import json
from dotenv import load_dotenv
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, ImageClip, CompositeVideoClip, AudioFileClip
print("MoviePy Imported")
import pyttsx3 as tts
print("pyttsx3 Imported")
import whisper
print("whisper Imported")
import requests
import google.generativeai as genai
print("GenerativeAI Imported")
load_dotenv()
from pyht import Client, TTSOptions, Format
print("PyHT Imported")
import torch
print("Torch Imported")
torch.device("cuda" if torch.cuda.is_available() else "cpu")
client = Client("fn8dNXoXdOPpVq9srLrsoLtiLLL2", os.getenv('HT_API_KEY'))
gemini_key = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=gemini_key)

MoviePy Imported
pyttsx3 Imported
whisper Imported
GenerativeAI Imported
PyHT Imported
Torch Imported


In [2]:
def generate_script(prompt, stories):
    model = genai.GenerativeModel('gemini-pro')
    default_prompt = """from now on return the output as regular text and the story should look
                    like it being narrated by someone. Write a Short and simple story,
                    You have to write maximum 170 words I want a plain text no symbols just the script, 
                    also the script content must be in 50 seconds duration, dont include any symbols or special characters,"""
    title_prompt = "For this story, I want you to write a 3 to 5 words title"

    for i in range(stories):
        story = model.generate_content(f"{default_prompt},{prompt}")
        title = model.generate_content(f"{title_prompt},{story}")
        
        print(f"Title: {title.text}")
        print(f"Story: {story.text}")

        json_data = json.dumps({"title": title.text, "story": story.text})
        with open(f"story_{i}.json", "w") as f:
            f.write(json_data)

In [3]:
generate_script("Mystery", 2)

Title: Haunted Mansion Visit
Story: In the eerie silence of the night, a chilling breeze whispered through the ancient trees. As I cautiously approached the dilapidated mansion, its windows boarded up like watchful eyes, a sense of unease washed over me.

With trembling hands, I pushed open the creaking door, its hinges groaning in protest. Inside, darkness enveloped me, suffocating and oppressive. As I fumbled for a light switch, my heart pounding in my chest, a sudden sound echoed from within the depths of the house—a faint laughter, as if from a long-forgotten soul.
Title: Unlocking Hidden Secrets
Story: In the eerie silence of the ancient manor, I found myself drawn to a dimly lit room at the end of a desolate corridor. Curiosity sparked within me, and I cautiously stepped inside.

Shadows danced on the walls, casting an ominous glow. Cobwebs hung heavy from the ceiling, and dust swirled in the faint breeze. As my eyes adjusted to the gloom, I noticed a small wooden box resting on 

In [4]:
def get_images(text):
    BASE = 'https://api.unsplash.com/photos/'
    PAYLOAD = {
        'query': text,
        'client_id': os.getenv('UNSPLASH_ACCESS_KEY')
    }
    response = requests.get(BASE, params=PAYLOAD)
    data = response.json()
    return data

In [5]:
# def eleven_speak(text_file, file_name):

#     voice_of = generate(text=text_file,
#                     voice = 'Adam')
#     play(voice_of)

def eleven_speak(text, file_name):
    engine = tts.init()
    engine.setProperty('rate', 200)
    engine.save_to_file(text, f'audio/{file_name}.mp3')
    engine.runAndWait()

for i in range(2):
    with open(f'story_{i}.json', 'r') as f:
        text = json.load(f)
        text = text['story']
        text = [word.replace('\n', '') for word in text]
        new_text = ''.join(text)
        print(new_text)
        eleven_speak(new_text, f'story_{i}')

In the eerie silence of the night, a chilling breeze whispered through the ancient trees. As I cautiously approached the dilapidated mansion, its windows boarded up like watchful eyes, a sense of unease washed over me.With trembling hands, I pushed open the creaking door, its hinges groaning in protest. Inside, darkness enveloped me, suffocating and oppressive. As I fumbled for a light switch, my heart pounding in my chest, a sudden sound echoed from within the depths of the house—a faint laughter, as if from a long-forgotten soul.
In the eerie silence of the ancient manor, I found myself drawn to a dimly lit room at the end of a desolate corridor. Curiosity sparked within me, and I cautiously stepped inside.Shadows danced on the walls, casting an ominous glow. Cobwebs hung heavy from the ceiling, and dust swirled in the faint breeze. As my eyes adjusted to the gloom, I noticed a small wooden box resting on an ornate table.A shiver ran down my spine as I reached out to open it. Inside,

In [6]:
def speech_to_text(audio_file):
    model = whisper.load_model("base")
    data = model.transcribe(audio_file, word_timestamps=True)

    start = [data['segments'][i]['words'][j]['start'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    end = [data['segments'][i]['words'][j]['end'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    text = [data['segments'][i]['words'][j]['word'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]

    return [start, end, text]

In [7]:
from moviepy.video.tools.subtitles import SubtitlesClip
from moviepy.editor import CompositeAudioClip, concatenate_audioclips

def generate_video():
    for i in range(2):
        with open(f'stories/story_{i}.txt', 'r') as file:
            print(f'Generating video for story {i}')
            text = file.read()
        
        words = text.split(' ')

        generator = lambda txt: TextClip(txt, 
                                    fontsize=70, 
                                    color='white', 
                                    bg_color='none', 
                                    font='Arial-Bold',
                                    method = 'caption',
                                    size = (1920, 1080))

        # def clip_generator():
        #     for _ in words:
        #         yield CompositeVideoClip([ImageClip('Z_1.jpg').set_duration(0.5).set_position('center')], size=(720, 1280))

        # clips = concatenate_videoclips(clip_generator(), method="compose")

        speech = speech_to_text(f'audio/story_{i}.mp3')
        print(f'Accessing speech for story {i}')
        subs = [((speech[0][k], speech[1][k]), speech[2][k]) for k in range(len(speech[0]))]

        subtitles = SubtitlesClip(subs, generator)
        audio_clip = AudioFileClip(f'audio/story_{i}.mp3')
        video = VideoFileClip('../assets/BG.mp4')
        final_complete = CompositeVideoClip([video.set_duration(audio_clip.duration), subtitles.set_position(('center', 'center'))])
        final_complete = final_complete.set_audio(audio_clip)
        final_complete.write_videofile(f"output_{i}.mp4" , fps=24, bitrate = '120000', preset = 'ultrafast', codec = 'h264_nvenc')
        final_complete.close()


In [8]:
generate_video()

Generating video for story 0
Accessing speech for story 0
Moviepy - Building video output_0.mp4.
MoviePy - Writing audio in output_0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_0.mp4



Moviepy - Done !
Moviepy - video ready output_0.mp4
Generating video for story 1
Accessing speech for story 1
Moviepy - Building video output_1.mp4.
MoviePy - Writing audio in output_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_1.mp4



Moviepy - Done !
Moviepy - video ready output_1.mp4


In [2]:
import httplib2
import os
import random
import sys
import time

from apiclient.discovery import build
from apiclient.errors import HttpError
from apiclient.http import MediaFileUpload
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client.tools import argparser, run_flow

In [3]:
# Explicitly tell the underlying HTTP transport library not to retry, since
# we are handling retry logic ourselves.
httplib2.RETRIES = 1

# Maximum number of times to retry before giving up.
MAX_RETRIES = 10

# Always retry when these exceptions are raised.
RETRIABLE_EXCEPTIONS = (httplib2.HttpLib2Error, IOError)

# Always retry when an apiclient.errors.HttpError with one of these status
# codes is raised.
RETRIABLE_STATUS_CODES = [500, 502, 503, 504]

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret. You can acquire an OAuth 2.0 client ID and client secret from
# the Google API Console at
# https://console.cloud.google.com/.
# Please ensure that you have enabled the YouTube Data API for your project.
# For more information about using OAuth2 to access the YouTube Data API, see:
#   https://developers.google.com/youtube/v3/guides/authentication
# For more information about the client_secrets.json file format, see:
#   https://developers.google.com/api-client-library/python/guide/aaa_client_secrets
CLIENT_SECRETS_FILE = "../secrets.json"


In [4]:
# This OAuth 2.0 access scope allows an application to upload files to the
# authenticated user's YouTube channel, but doesn't allow other types of access.
YOUTUBE_UPLOAD_SCOPE = "https://www.googleapis.com/auth/youtube.upload"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [9]:
VALID_PRIVACY_STATUSES = ("public", "private", "unlisted")


def get_authenticated_service(args):
    flow = flow_from_clientsecrets(CLIENT_SECRETS_FILE,
                                   scope=YOUTUBE_UPLOAD_SCOPE,
                                   message=MISSING_CLIENT_SECRETS_MESSAGE)

    storage = Storage("%s-oauth2.json" % sys.argv[0])
    credentials = storage.get()

    if credentials is None or credentials.invalid:
        credentials = run_flow(flow, storage, args)

    return build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                 http=credentials.authorize(httplib2.Http()))
    
MISSING_CLIENT_SECRETS_MESSAGE = """
WARNING: Please configure OAuth 2.0

To make this sample run you will need to populate the client_secrets.json file
found at:

   %s

with information from the API Console
https://console.cloud.google.com/

For more information about the client_secrets.json file format, please visit:
https://developers.google.com/api-client-library/python/guide/aaa_client_secrets
""" 

In [10]:
def initialize_upload(youtube, options):
    tags = None
    if options.keywords:
        tags = options.keywords.split(",")

    body = dict(
        snippet=dict(
            title=options.title,
            description=options.description,
            tags=tags,
            categoryId=options.category
        ),
        status=dict(
            privacyStatus=options.privacyStatus
        )
    )

    # Call the API's videos.insert method to create and upload the video.
    insert_request = youtube.videos().insert(
        part=",".join(body.keys()),
        body=body,
        # The chunksize parameter specifies the size of each chunk of data, in
        # bytes, that will be uploaded at a time. Set a higher value for
        # reliable connections as fewer chunks lead to faster uploads. Set a lower
        # value for better recovery on less reliable connections.
        #
        # Setting "chunksize" equal to -1 in the code below means that the entire
        # file will be uploaded in a single HTTP request. (If the upload fails,
        # it will still be retried where it left off.) This is usually a best
        # practice, but if you're using Python older than 2.6 or if you're
        # running on App Engine, you should set the chunksize to something like
        # 1024 * 1024 (1 megabyte).
        media_body=MediaFileUpload(options.file, chunksize=-1, resumable=True)
    )

    resumable_upload(insert_request)

# This method implements an exponential backoff strategy to resume a
# failed upload.


def resumable_upload(insert_request):
    response = None
    error = None
    retry = 0
    while response is None:
        try:
            print("Uploading file...")
            status, response = insert_request.next_chunk()
            if response is not None:
                if 'id' in response:
                    print("Video id '%s' was successfully uploaded." %
                          response['id'])
                else:
                    exit("The upload failed with an unexpected response: %s" % response)
        except HttpError as e:
            if e.resp.status in RETRIABLE_STATUS_CODES:
                error = "A retriable HTTP error %d occurred:\n%s" % (e.resp.status,
                                                                     e.content)
            else:
                raise
        except RETRIABLE_EXCEPTIONS as e:
            error = "A retriable error occurred: %s" % e

        if error is not None:
            print(error)
            retry += 1
            if retry > MAX_RETRIES:
                exit("No longer attempting to retry.")

            max_sleep = 2 ** retry
            sleep_seconds = random.random() * max_sleep
            print("Sleeping %f seconds and then retrying..." % sleep_seconds)
            time.sleep(sleep_seconds)

In [11]:
if __name__ == '__main__':
    argparser.add_argument("--file", required=True,
                           help="Video file to upload")
    argparser.add_argument("--title", help="Video title", default="Test Title")
    argparser.add_argument("--description", help="Video description",
                           default="Test Description")
    argparser.add_argument("--category", default="22",
                           help="Numeric video category. " +
                           "See https://developers.google.com/youtube/v3/docs/videoCategories/list")
    argparser.add_argument("--keywords", help="Video keywords, comma separated",
                           default="")
    argparser.add_argument("--privacyStatus", choices=VALID_PRIVACY_STATUSES,
                           default=VALID_PRIVACY_STATUSES[0], help="Video privacy status.")
    args = argparser.parse_args()

    if not os.path.exists(args.file):
        exit("Please specify a valid file using the --file= parameter.")

    youtube = get_authenticated_service(args)
    try:
        initialize_upload(youtube, args)
    except HttpError as e:
        print("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))

ArgumentError: argument --file: conflicting option string: --file